# Task 1

### Data processing

In [1]:
pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Download raw data
data = pd.read_csv('data/Groceries data train.csv')

In [4]:
data

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,30/01/2015,frankfurter,2015,1,30,4
1,1292,24/10/2015,pork,2015,10,24,5
2,4206,4/04/2014,root vegetables,2014,4,4,4
3,4369,25/08/2015,onions,2015,8,25,1
4,1522,1/07/2014,waffles,2014,7,1,1
...,...,...,...,...,...,...,...
26995,4206,14/01/2014,rolls/buns,2014,1,14,1
26996,1660,15/06/2014,domestic eggs,2014,6,15,6
26997,1595,10/05/2015,liver loaf,2015,5,10,6
26998,3231,31/03/2015,sausage,2015,3,31,1


In [5]:
data.columns

Index(['Member_number', 'Date', 'itemDescription', 'year', 'month', 'day',
       'day_of_week'],
      dtype='object')

In [6]:
# Group the data by 'Member_number' and 'Date', then aggregate the 'itemDescription' into lists
transactions_data = data.groupby(['Member_number'])['itemDescription'].apply(list).reset_index()

# Extract the list of transactions
transactions = transactions_data['itemDescription'].tolist()

sorted_transactions = [sorted(list(set(transaction))) for transaction in transactions]

transactions = sorted_transactions

transactions = [transaction for transaction in transactions if len(transaction) > 1]

# Display the first transaction as an example
transactions[0]

['canned beer',
 'misc. beverages',
 'pastry',
 'pickled vegetables',
 'sausage',
 'soda',
 'yogurt']

In [7]:
print(type(transactions))

<class 'list'>


### Mine frequent patterns by using Apriori algorithm

In [8]:
pip install apyori

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
from apyori import apriori
from mlxtend.frequent_patterns import association_rules
rules = apriori(transactions, min_support=0.002, min_confidence=0.8,min_lift=1, min_length=1)
Results = list(rules)
Results

[RelationRecord(items=frozenset({'whole milk', 'bottled water', 'canned vegetables'}), support=0.002425222312045271, ordered_statistics=[OrderedStatistic(items_base=frozenset({'bottled water', 'canned vegetables'}), items_add=frozenset({'whole milk'}), confidence=0.8181818181818182, lift=2.2760665121984465)]),
 RelationRecord(items=frozenset({'whole milk', 'spices', 'bottled water'}), support=0.002425222312045271, ordered_statistics=[OrderedStatistic(items_base=frozenset({'spices', 'bottled water'}), items_add=frozenset({'whole milk'}), confidence=0.8181818181818182, lift=2.2760665121984465)]),
 RelationRecord(items=frozenset({'dishes', 'butter', 'whole milk'}), support=0.002155753166262463, ordered_statistics=[OrderedStatistic(items_base=frozenset({'dishes', 'butter'}), items_add=frozenset({'whole milk'}), confidence=1.0, lift=2.7818590704647677)]),
 RelationRecord(items=frozenset({'whole milk', 'newspapers', 'canned fish'}), support=0.002425222312045271, ordered_statistics=[OrderedSt

In [19]:
df_results = pd.DataFrame(Results)
df_results

,items,support,ordered_statistics
0,"(whole milk, bottled water, canned vegetables)",0.002425,"[((bottled water, canned vegetables), (whole m..."
1,"(whole milk, spices, bottled water)",0.002425,"[((spices, bottled water), (whole milk), 0.818..."
2,"(dishes, butter, whole milk)",0.002156,"[((dishes, butter), (whole milk), 1.0, 2.78185..."
3,"(whole milk, newspapers, canned fish)",0.002425,"[((newspapers, canned fish), (whole milk), 0.8..."
4,"(whole milk, napkins, white bread)",0.004581,"[((napkins, white bread), (whole milk), 0.8095..."
...,...,...,...
81,"(bottled water, pip fruit, whole milk, yogurt,...",0.002695,"[((yogurt, other vegetables, bottled water, pi..."
82,"(bottled water, whole milk, rolls/buns, soda, ...",0.002425,"[((rolls/buns, bottled water, whipped/sour cre..."
83,"(whole milk, shopping bags, canned beer, rolls...",0.002695,"[((rolls/buns, shopping bags, soda, canned bee..."
84,"(yogurt, other vegetables, frozen vegetables, ...",0.002425,"[((yogurt, other vegetables, frozen vegetables..."


In [20]:
# keep support in a separate data frame so we can use later
support = df_results.support

In [21]:
#all four empty list which will contain lhs, rhs, confidance and lift respectively.
first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list
# first and second element was frozenset which need to be converted in list
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [22]:
# convert all four list into dataframe for further operation
lhs = pd.DataFrame(first_values)
rhs = pd.DataFrame(second_values)

confidance=pd.DataFrame(third_values,columns=['Confidance'])

lift=pd.DataFrame(fourth_value,columns=['lift'])

In [23]:
# concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final
df_final.fillna(value=' ', inplace=True)
df_final.head()

,0,1,2,3,0,support,Confidance,lift
0,bottled water,canned vegetables,,,whole milk,0.002425,0.818182,2.276067
1,spices,bottled water,,,whole milk,0.002425,0.818182,2.276067
2,dishes,butter,,,whole milk,0.002156,1.000000,2.781859
3,newspapers,canned fish,,,whole milk,0.002425,0.818182,2.276067
4,napkins,white bread,,,whole milk,0.004581,0.809524,2.251981


In [24]:
#set column name
df_final.columns = ['lhs',1,2,3,'rhs','support','confidance','lift']
df_final.head()

,lhs,1,2,3,rhs,support,confidance,lift
0,bottled water,canned vegetables,,,whole milk,0.002425,0.818182,2.276067
1,spices,bottled water,,,whole milk,0.002425,0.818182,2.276067
2,dishes,butter,,,whole milk,0.002156,1.000000,2.781859
3,newspapers,canned fish,,,whole milk,0.002425,0.818182,2.276067
4,napkins,white bread,,,whole milk,0.004581,0.809524,2.251981


In [25]:
# add all three column to lhs itemset only
df_final['lhs'] = df_final['lhs'] + str(", ")  + df_final[1] + df_final[2] + df_final[3]

df_final['rhs'] = df_final['rhs'] + str(", ")

In [26]:
df_final.drop(columns=[1,2,3],inplace=True)

In [27]:
df_final.sort_values('lift', ascending=False).head(10)

,lhs,rhs,support,confidance,lift
27,"whipped/sour cream, bottled waterwaffles","tropical fruit,",0.002156,0.888889,5.294810
21,"bottled water, canned beerspecialty chocolate","root vegetables,",0.002156,0.888889,5.051557
32,"whole milk, candypork","yogurt,",0.002156,0.888889,4.175527
77,"whole milk, other vegetablesberriestropical fruit","yogurt,",0.002425,0.818182,3.843383
6,"frankfurter, sodaUHT-milk","rolls/buns,",0.002156,1.000000,3.666996
59,"frankfurter, sausagepastry","soda,",0.002425,0.818182,3.362428
35,"domestic eggs, shopping bagscanned beer","soda,",0.002425,0.818182,3.362428
84,"yogurt, other vegetablesfrozen vegetablessausage","rolls/buns,",0.002425,0.900000,3.300296
44,"citrus fruit, frozen mealswhole milk","rolls/buns,",0.002695,0.833333,3.055830
60,"frozen meals, frozen vegetableswhole milk","other vegetables,",0.002156,0.888889,3.037446


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
df

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

frequent_itemsets = apriori(df, min_support=0.002, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules

In [ ]:
# frequent_itemsets = fpgrowth(df, min_support=0.002, use_colnames=True)
# frequent_itemsets

# from mlxtend.frequent_patterns import association_rules
# rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
# rules

In [ ]:
def predict_items(purchased_items):
    # create a list to store the predicted items
    predicted_items = []
    
    # iterate over the top rules
    for index, row in rules.iterrows():
        # get the items in the antecedent and consequent of the rule
        antecedent = row["antecedents"]
        consequent = row["consequents"]
        
        # check if all the items in the antecedent are in the purchased items
        if antecedent.issubset(purchased_items):
            # add the items in the consequent to the predicted items
            for item in consequent:
                if item not in purchased_items and item not in predicted_items:
                    predicted_items.append(item)
    
    # return the list of predicted items
    return predicted_items

In [ ]:
# make a prediction for a sample set of purchased items
purchased_items = {"yogurt", "soda"}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)

In [ ]:
# make a prediction for a sample set of purchased items
purchased_items = {'canned beer',
 'misc. beverages',
 'pastry',
 'pickled vegetables',
 'sausage',
 'soda',
 'yogurt'}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)